In [42]:
import os 

In [ ]:
%pwd

In [ ]:
%ls

In [ ]:
# os.chdir("/datascience-project-wine-quality")
# %pwd

In [50]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [51]:
from constants import *
from utils import read_yaml, create_directories

In [52]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(PARAMS_FILE_PATH)
        self.schema= read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root]) ## creating artifacts folder

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config= self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir 
        )
        return data_ingestion_config

        

### Component 1: Data Ingestion

In [53]:
import os 
import urllib.request as request
import logger
import zipfile

In [54]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config

    ## Downloading the zip data file
    def doownload_zip_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} dowloaded!! with following info: \n{headers}")
        else:
            logger.info(f"File already exists.")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [55]:
## lets use above component!!

try: 
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(data_ingestion_config)
    data_ingestion.doownload_zip_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-25 20:19:52,191: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-25 20:19:52,195: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-25 20:19:52,195: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-25 20:19:52,195: INFO: common: created directory at: artifacts]
[2024-12-25 20:19:52,200: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-25 20:19:53,336: INFO: 3996754210: artifacts/data_ingestion/data.zip dowloaded!! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DF19:4FD78:9CB12:E8F83:676C1B91
Accept-Ranges: bytes
Date: Wed, 25 